# 04 - Test du serving MLflow

Ce notebook valide le modèle servi via Docker/MLflow.

## Prérequis

1. Construire l'image : `docker build -t credit-scoring-model .`
2. Lancer le conteneur : `docker run -p 1234:1234 credit-scoring-model`
3. Vérifier que le serveur répond : `curl http://localhost:1234/health`

## Notes importantes

- Le dataset de test est déjà préparé (même format que lors de l'entraînement)
- Seule conversion nécessaire : les colonnes lues comme `object` par pandas doivent être converties en numériques
- Les NaN sont conservés (LightGBM les supporte nativement)
- Aucun autre retraitement n'est nécessaire



In [28]:
import pandas as pd
import requests
import json
import sys
import os
import numpy as np
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.model_utils import clean_feature_names
from src.metrics import custom_business_cost

## 2. Chargement du (Données de Test)

Pour évaluer le modèle en conditions réelles tout en pouvant calculer des métriques, nous utilisons le **sample de 10 000 lignes** isolé lors de l'étape de préparation des données.

* **`test_sample_features.csv`** : Les caractéristiques clients (ce qu'on envoie à l'API).
* **`test_sample_labels.csv`** : La vérité terrain (TARGET), gardée en local pour vérifier la réponse de l'API.

In [ ]:
URL = "http://localhost:1234/invocations"
FEATURES_PATH = "../datasets/final/test_sample_features.csv"
LABELS_PATH = "../datasets/final/test_sample_labels.csv"
IDS_PATH = "../datasets/final/test_sample_ids.csv"
SEUIL_OPTIMAL = 0.5

df_features = pd.read_csv(FEATURES_PATH, keep_default_na=True, na_values=['', 'NULL', 'N/A', 'null', 'None'])
df_labels = pd.read_csv(LABELS_PATH)
df_ids = pd.read_csv(IDS_PATH)

print(f"Données chargées : {df_features.shape[0]} échantillons, {df_features.shape[1]} features")

Données chargées : 10000 échantillons, 1845 features


## 3. Prétraitement et Sérialisation JSON

Le modèle hébergé dans Docker (via MLflow) est strict sur les formats d'entrée. Nous devons reproduire les transformations effectuées avant l'entraînement :

1.  **Nettoyage des noms de colonnes :** Suppression des caractères spéciaux (via `clean_feature_names`).
2.  **Suppression des IDs :** Le modèle ne prend pas `SK_ID_CURR` en entrée.
3.  **Typage fort :** Conversion explicite de toutes les données en numérique (`float64`) pour éviter les erreurs d'interprétation du JSON par le serveur (erreur `arrays of bytes/strings`).
4.  **Formatage :** Conversion en JSON au format `split` (orienté colonnes) pour l'envoi HTTP.

In [ ]:
X_clean = clean_feature_names(df_features)

cols_to_drop = ['SK_ID_CURR', 'index', 'Unnamed: 0'] 
X_final = X_clean.drop(columns=[c for c in cols_to_drop if c in X_clean.columns])

for col in X_final.columns:
    if X_final[col].dtype == 'object':
        X_final[col] = pd.to_numeric(X_final[col], errors='coerce')
    elif X_final[col].dtype == 'bool':
        X_final[col] = X_final[col].astype(int)
    if X_final[col].dtype in [np.int64, np.int32, np.int16, np.int8]:
        X_final[col] = X_final[col].astype(np.float64)

X_final = X_final.replace([np.inf, -np.inf], np.nan)

json_payload = json.loads(X_final.to_json(orient="split", double_precision=15))
payload = {"dataframe_split": json_payload}

## 4. Appel API et Analyse de la Performance

Nous simulons ici l'envoi en masse de dossiers de crédit au serveur.

1.  Envoi d'une requête `POST` à `http://localhost:1234/invocations`.
2.  Récupération des probabilités de défaut renvoyées par le modèle Docker.
3.  Comparaison avec les vrais labels (`y_true`) pour calculer :
    * **L'AUC :** Performance globale du modèle.
    * **Le Coût Métier :** Impact financier (10 * Faux Négatifs + 1 * Faux Positifs).
    * **La Matrice de Confusion :** Détail des erreurs.

In [ ]:
response = requests.post(URL, json=payload, timeout=30)

if response.status_code == 200:
    response_data = response.json()
    predictions = response_data.get('predictions', response_data) if isinstance(response_data, dict) else response_data
    
    y_prob = []
    for p in predictions:
        if isinstance(p, list):
            score = float(p[1]) if len(p) > 1 else float(p[0]) 
        else:
            score = float(p)
        y_prob.append(score)
    
    y_prob = np.array(y_prob)
    y_true = df_labels['TARGET'].values
    
    auc_score = roc_auc_score(y_true, y_prob)
    y_pred_bin = (y_prob >= SEUIL_OPTIMAL).astype(int)
    cout_total = custom_business_cost(y_true, y_pred_bin)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_bin).ravel()
    
    print("Résultats du serving")
    print(f"\nAUC Score : {auc_score:.4f}")
    print(f"Coût Métier Total : {cout_total:,}")
    print(f"\nMatrice de Confusion :")
    print(f"   - Vrais Négatifs : {tn:,}")
    print(f"   - Vrais Positifs : {tp:,}")
    print(f"   - Faux Positifs  : {fp:,}  (Coût : {fp:,})")
    print(f"   - Faux Négatifs  : {fn:,}  (Coût : {fn*10:,})")
    print(f"\nRapport de Classification :")
    print(classification_report(y_true, y_pred_bin, target_names=['Bons payeurs', 'Mauvais Payeurs']))
    
    print("Exemples de décisions pour 5 clients")
    print(f"{'ID Client':<12} | {'Score':<10} | {'Décision':<12} | {'Réalité':<10} | {'Check'}")
    ids = df_ids['SK_ID_CURR'].values
    for i in range(min(5, len(ids))):
        dec = "REFUSÉ" if y_pred_bin[i] == 1 else "ACCORDÉ"
        real = "DÉFAUT" if y_true[i] == 1 else "OK"
        check = "1" if y_pred_bin[i] == y_true[i] else "0"
        print(f"{ids[i]:<12} | {y_prob[i]:.4f}     | {dec:<12} | {real:<10} | {check}")
else:
    print(f"Erreur API ({response.status_code}) : {response.text}")

Résultats du serving

AUC Score : 0.7160
Coût Métier Total : 4,880

Matrice de Confusion :
   - Vrais Négatifs : 6,773
   - Vrais Positifs : 561
   - Faux Positifs  : 2,420  (Coût : 2,420)
   - Faux Négatifs  : 246  (Coût : 2,460)

Rapport de Classification :
                 precision    recall  f1-score   support

   Bons payeurs       0.96      0.74      0.84      9193
Mauvais Payeurs       0.19      0.70      0.30       807

       accuracy                           0.73     10000
      macro avg       0.58      0.72      0.57     10000
   weighted avg       0.90      0.73      0.79     10000

Exemples de décisions pour 5 clients
ID Client    | Score      | Décision     | Réalité    | Check
374942       | 0.0000     | ACCORDÉ      | OK         | 1
449334       | 0.0000     | ACCORDÉ      | OK         | 1
308306       | 0.0000     | ACCORDÉ      | OK         | 1
431049       | 0.0000     | ACCORDÉ      | OK         | 1
208379       | 0.0000     | ACCORDÉ      | OK         | 1


# Analyse
**AUC**: On se retrouve avec un AUC bien plus faible qu'à l'entrainement et la validation, probablement car notre jeu de test est insuffisant. 